In [1]:
from for_sale_env_r1.fs_first_round import env as fs_env
import numpy as np

In [2]:
env = fs_env()
env.reset()
for i, agent in enumerate(env.agent_iter()):
    observation, reward, done, info = env.last()
    if i == 0:
        action = 5
        env.step(action)
    elif i == 3:
        break
    else:
        env.step(0)
    
    
print(len(observation["observation"]))
print(observation["observation"])
env.render()

42
[13.         8.         0.         0.         1.         0.
 13.         1.         3.        18.         4.         0.
  0.         1.         0.        18.         3.         2.
 18.         6.         0.         0.         1.         0.
 18.         9.         1.        11.        10.         9.
  7.         5.         3.         2.         1.         0.
  0.         0.         9.         3.         1.         3.3993464]

highest bid:  {'agent': 'player_2', 'value': 0}
current bids by each player
made by player:  1
current agents in game:  deque(['player_0', 'player_1', 'player_2'])
The agent to move:  player_0
board:  [9, 3, 1]
stack:  [11, 10, 9, 7, 5, 3, 2, 1, 0, 0, 0]
coins: {'player_0': 13, 'player_1': 18, 'player_2': 18}
cards: {'player_0': [8], 'player_1': [4], 'player_2': [6]}



In [93]:
(np.std([1,11]) - np.std([1])) / 2
sum([5, 4, 4, 4, 1, 5, 5, 4, 2]) * 3 + 11 + 4 + 4 + 4 + 1

126

In [4]:
# 3 player round played with 9 + 2 possible cards
# the standard deviation decides the final bit
# counting the total amount of bits needed for representation decides if onehot or binary representation, less is best
curr_obs = observation["observation"]
np.array(list(bin(int(curr_obs[0]))[2:]), dtype="float32")
print(bin(11))
print(len(curr_obs))
"{0:05b}".format(3)

# list('0001100')

0b1011
42


'00011'

In [98]:
def convert_3_rounds_fs1_obs_to_bin_obs(obs):
    stack_len = 11
    rounds = 3
    
    coin_numbers = [0, 5, 6, 9, 14, 15, 18, 23, 24]
    board_card_numbers = [1,2,3,7,10,11,12,16,19,20,21,25,38,39,40]
    can_bet_numbers = [4,13,22]
    order_numbers = [8,17,26]
    std_number = 41
    bin_rep = []
    onehot_of_stack = []
    for i, num in enumerate(obs):
        if i in coin_numbers:
            bin_rep.extend(list("{0:05b}".format(int(num))))
        elif i in board_card_numbers:
            bin_rep.extend(list("{0:04b}".format(int(num))))
        elif i in can_bet_numbers:
            bin_rep.append(int(num))
        elif i in order_numbers:
            bin_rep.extend(list("{0:02b}".format(int(num))))
        elif i == std_number:
            if num > (np.std([1,11]) - np.std([1])) / 2:
                bin_rep.append(1)
            else:
                bin_rep.append(0)
        else:
            onehot = np.zeros(stack_len, dtype="float32")
            if int(num) != 0:
                onehot[1-int(num)] = 1
            onehot_of_stack.append(onehot)
            
    bin_rep.extend(sum(onehot_of_stack))
    bin_rep = np.hstack(bin_rep)
    return bin_rep.astype(np.float32)
            
    
def convert_3_rounds_fs1_obs_to_bin_obs(obs):
    # this one works
    stack_len = 11
    rounds = 3
    
    coin_numbers = [0, 5, 6, 9, 14, 15, 18, 23, 24]
    board_card_numbers = [1,2,3,7,10,11,12,16,19,20,21,25,38,39,40]
    can_bet_numbers = [4,13,22]
    order_numbers = [8,17,26]
    std_number = 41
    last_stack_nr = 37
    bin_rep = []
    onehot_of_stack = np.zeros(stack_len, dtype="float32")
    for i, num in enumerate(obs):
        if i in coin_numbers:
            bin_rep.extend(list("{0:05b}".format(int(num))))
        elif i in board_card_numbers:
            bin_rep.extend(list("{0:04b}".format(int(num))))
        elif i in can_bet_numbers:
            bin_rep.append(int(num))
        elif i in order_numbers:
            bin_rep.extend(list("{0:02b}".format(int(num))))
        elif i == std_number:
            if num > (np.std([1,11]) - np.std([1])) / 2:
                bin_rep.append(1)
            else:
                bin_rep.append(0)
            
        else:
            if i == last_stack_nr:
                # print(int(num))
                if int(num) != 0:
                    onehot_of_stack[-int(num)] = 1
                    bin_rep.extend(list(onehot_of_stack))
                    # print(onehot_of_stack)
                    # print("WOW")
                else:
                    onehot_of_stack[-int(num)] = 1
                    bin_rep.extend(list(onehot_of_stack))
                    
            else:
                # print(int(num))
                if int(num) != 0:
                    onehot_of_stack[-int(num)] = 1
            
    bin_rep = np.hstack(bin_rep)
    return bin_rep.astype(np.float32)

convert_3_rounds_fs1_obs_to_bin_obs(curr_obs).shape

(126,)

In [104]:
print(curr_obs)
print(len(curr_obs))

bin_obs = convert_3_rounds_fs1_obs_to_bin_obs(curr_obs)
print(bin_obs)
def bin_numpy_to_number(bin_array):
    bin_str = ""
    if type(bin_array) == np.float32:
        bin_str += str(int(bin_array))        
    else:
        for digit in bin_array:
            bin_str += str(int(digit))
    dec_num = int(bin_str, base=2)
    return dec_num

def reverse_onehot(target, stack_len):
    stack = np.zeros(stack_len)
    actives = np.argwhere(target == 1)
    for i, act in enumerate(actives):
        stack[i] = stack_len - act
    return stack

    
def convert_3_rounds_fs1_bin_obs_to_obs(obs):
    stack_len = 11
    new_obs = np.zeros(42)
    new_obs[0] = bin_numpy_to_number(obs[0:5])
    new_obs[1] = bin_numpy_to_number(obs[5:9])
    new_obs[2] = bin_numpy_to_number(obs[9:13])
    new_obs[3] = bin_numpy_to_number(obs[13:17])
    new_obs[4] = bin_numpy_to_number(obs[17])
    new_obs[5] = bin_numpy_to_number(obs[18:23])
    new_obs[6] = bin_numpy_to_number(obs[23:28])
    new_obs[7] = bin_numpy_to_number(obs[28:32])
    new_obs[8] = bin_numpy_to_number(obs[32:34])
    new_obs[9] = bin_numpy_to_number(obs[34:39])
    new_obs[10] = bin_numpy_to_number(obs[39:43])
    new_obs[11] = bin_numpy_to_number(obs[43:47])
    new_obs[12] = bin_numpy_to_number(obs[47:51])
    new_obs[13] = bin_numpy_to_number(obs[51])
    new_obs[14] = bin_numpy_to_number(obs[52:57])
    new_obs[15] = bin_numpy_to_number(obs[57:62])
    new_obs[16] = bin_numpy_to_number(obs[62:66])
    new_obs[17] = bin_numpy_to_number(obs[66:68])
    new_obs[18] = bin_numpy_to_number(obs[68:73])
    new_obs[19] = bin_numpy_to_number(obs[73:77])
    new_obs[20] = bin_numpy_to_number(obs[77:81])
    new_obs[21] = bin_numpy_to_number(obs[81:85])
    new_obs[22] = bin_numpy_to_number(obs[85])
    new_obs[23] = bin_numpy_to_number(obs[86:91])
    new_obs[24] = bin_numpy_to_number(obs[91:96])
    new_obs[25] = bin_numpy_to_number(obs[96:100])
    new_obs[26] = bin_numpy_to_number(obs[100:102])
    new_obs[27:38] = reverse_onehot(obs[102:113], stack_len)
    new_obs[38] = bin_numpy_to_number(obs[113:117])
    new_obs[39] = bin_numpy_to_number(obs[117:121])
    new_obs[40] = bin_numpy_to_number(obs[121:125])
    new_obs[41] = np.std(new_obs[38:41])
    
    return np.array(new_obs, dtype="float32")

ret_obs = convert_3_rounds_fs1_bin_obs_to_obs(bin_obs)
print("ret obs are here:")
print(ret_obs)
curr_obs == ret_obs

[13.         8.         0.         0.         1.         0.
 13.         1.         3.        18.         4.         0.
  0.         1.         0.        18.         3.         2.
 18.         6.         0.         0.         1.         0.
 18.         9.         1.        11.        10.         9.
  7.         5.         3.         2.         1.         0.
  0.         0.         9.         3.         1.         3.3993464]
42
[0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1.
 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0.
 1. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1. 0. 0. 1.
 1. 0. 0. 0. 1. 1.]
ret obs are here:
[13.         8.         0.         0.         1.         0.
 13.         1.         3.        18.         4.         0.
  0.         1.         0.        18.         

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [118]:
def make_observation_into_gamestate(obs):
    # own is first, then next player, then final player
    new_obs = obs.astype(int)
    print("Player to move coins: {}".format(new_obs[0]))
    print("Player to move cards: {}".format(new_obs[1:4]))
    print("Player to move can bid: {}".format(bool(new_obs[4])))
    print("Player to move current bid: {}".format(new_obs[5]))
    print("Player to move coins on fold: {}".format(new_obs[6]))
    print("Player to move achieved card on fold: {}".format(new_obs[7]))
    print("Player to move moving order if all folds now: {}".format(new_obs[8]))
    print("Next player coins: {}".format(new_obs[9]))
    print("Next player cards: {}".format(new_obs[10:13]))
    print("Next player can bid: {}".format(bool(new_obs[13])))
    print("Next player current bid: {}".format(new_obs[14]))
    print("Next player coins on fold: {}".format(new_obs[15]))
    print("Next player achieved card on fold: {}".format(new_obs[16]))
    print("Next player moving order if all folds now: {}".format(new_obs[17]))
    print("Final player coins: {}".format(new_obs[18]))
    print("Final player cards: {}".format(new_obs[19:22]))
    print("Final player can bid: {}".format(bool(new_obs[22])))
    print("Final player current bid: {}".format(new_obs[23]))
    print("Final player coins on fold: {}".format(new_obs[24]))
    print("Final player card on fold: {}".format(new_obs[25]))
    print("Final player moving order if all folds now: {}".format(new_obs[26]))
    print("Current stack: {}".format(new_obs[27:38]))
    print("Current board {}".format(new_obs[38:41]))
    
    
    
    
    
    
    
    

make_observation_into_gamestate(ret_obs)

Player to move coins: 13
Player to move cards: [8 0 0]
Player to move can bid: True
Player to move current bid: 0
Player to move coins on fold: 13
Player to move achieved card on fold: 1
Player to move moving order if all folds now: 3
Next player coins: 18
Next player cards: [4 0 0]
Next player can bid: True
Next player current bid: 0
Next player coins on fold: 18
Next player achieved card on fold: 3
Next player moving order if all folds now: 2
Final player coins: 18
Final player cards: [6 0 0]
Final player can bid: True
Final player current bid: 0
Final player coins on fold: 18
Final player card on fold: 9
Final player moving order if all folds now: 1
Current stack: [11 10  9  7  5  3  2  1  0  0  0]
Current board [9 3 1]


101